In [1]:
# Import de librairies 
import pandas as pd 
import numpy as np

In [2]:
# Import de fichiers log et dataBase 
log_file = 'data/log.csv'
DataBase_file = 'data/DataBase.csv'
log_data = pd.read_csv(log_file, delimiter=";")
dataBase_data = pd.read_csv(DataBase_file, delimiter=";")

In [3]:
#Supression de toutes les lignes dont le idUser est null ou vide
df = log_data.dropna(subset=['idUser'])

In [4]:
"""Cette fonction permet de regrouper les information client 
d’une colonne par apport à une autre colonne. Par exemple grouper 
les actions réalisées par client"""

def gourb_Column_B_by_Column_A(dataFrame, Column_A_Name, Column_B_Name):
	liste = []
	columns = [Column_A_Name,"List of " + Column_B_Name,"Number of " + Column_B_Name]
	for id_user in set(list(dataFrame[Column_A_Name])):
		user_elements = list(dataFrame[dataFrame[Column_A_Name] == id_user][Column_B_Name])
		liste.append((int(id_user),user_elements,len(user_elements)))
	return pd.DataFrame(liste, columns = columns) 

# Liste des cookie / idUser

In [5]:
df_gourb_cookie_by_idUser = gourb_Column_B_by_Column_A(df,'idUser','cookie')
#j'afficherai que les  10 premiàres lignes à titre d'exemple
df_gourb_cookie_by_idUser.head(10)

,idUser,List of cookie,Number of cookie
0,9083,[ts8uLmkYCt],1
1,8,[h9lGXl93mh],1
2,401,"[682f2zlRQW, 682f2zlRQW, 682f2zlRQW, 682f2zlRQ...",19
3,19,"[mcUSqojj5N, mcUSqojj5N]",2
4,13846,[HcRZNKwvVc],1
5,15153,"[L2urYBOcoH, L2urYBOcoH, L2urYBOcoH, L2urYBOco...",21
6,15158,[bv7CK2ymXD],1
7,8381,"[zDxzTwul8d, zDxzTwul8d, zDxzTwul8d, zDxzTwul8...",12
8,15168,"[OhQVP0mgJa, 80nvNTn36e, 80nvNTn36e]",3
9,15428,"[nNmCqB2w4c, nNmCqB2w4c, nNmCqB2w4c, nNmCqB2w4...",6


# Liste des actions propres à chaque utilisateur et le nombre d'action de chaque utilisateur

In [6]:
df_gourb_typeAction_by_idUser = gourb_Column_B_by_Column_A(df,'idUser','typeAction')
#j'afficherai que les  10 premiàres lignes à titre d'exemple
# Nous listons aussi pour chaque utilisateur le nombre d'action réalisée
df_gourb_typeAction_by_idUser.head(10)

,idUser,List of typeAction,Number of typeAction
0,9083,[5],1
1,8,[37],1
2,401,"[37, 37, 37, 35, 51, 37, 51, 37, 51, 5, 37, 37...",19
3,19,"[5, 37]",2
4,13846,[5],1
5,15153,"[37, 37, 51, 37, 51, 37, 51, 37, 37, 51, 37, 5...",21
6,15158,[37],1
7,8381,"[5, 37, 51, 35, 37, 51, 37, 51, 37, 51, 37, 51]",12
8,15168,"[37, 37, 37]",3
9,15428,"[5, 37, 49, 57, 54, 55]",6


In [11]:
df1 = df_gourb_typeAction_by_idUser

# Définition de segment par utilisateur

In [12]:
#Elle permet de convertir une liste d'action en points
def All_type_action_to_points(list_of_actions):
	points = 0
	for action  in list_of_actions:
		points += int(action)//10
	return points

In [13]:
#conversion de point en segment 
def points_segment(liste):
	segment = All_type_action_to_points(liste)
	if segment < 5 :
		return 1
	elif segment < 10 :
		return 2
	elif segment < 15 :
		return 3
	elif segment < 20:
		return 4
	else:
		return 5

In [14]:
columns1 = ['idUser',"List of typeAction","Number of typeAction","Segment"]
liste = []
for id_user in list(df1['idUser']):
	row = df1[df1['idUser'] == id_user]
	list_of_actions = list(np.array(row["List of typeAction"])[0])
	segment = points_segment(list_of_actions)
	liste.append((int(id_user),list_of_actions,len(list_of_actions),segment))
df_with_segment = pd.DataFrame(liste, columns = columns1)

In [15]:
#j'afficherai que les  10 premiàres lignes à titre d'exemple
df_with_segment.head(10)

,idUser,List of typeAction,Number of typeAction,Segment
0,9083,[5],1,1
1,8,[37],1,1
2,401,"[37, 37, 37, 35, 51, 37, 51, 37, 51, 5, 37, 37...",19,5
3,19,"[5, 37]",2,1
4,13846,[5],1,1
5,15153,"[37, 37, 51, 37, 51, 37, 51, 37, 37, 51, 37, 5...",21,5
6,15158,[37],1,1
7,8381,"[5, 37, 51, 35, 37, 51, 37, 51, 37, 51, 37, 51]",12,5
8,15168,"[37, 37, 37]",3,2
9,15428,"[5, 37, 49, 57, 54, 55]",6,5


# Association de  l'utilisateur à son mail  et le nombre d'action réalisé pour chaque compte mail

In [17]:
df2 = df_with_segment

In [18]:
dataBase_data.head(3)

,IdUser,mail
0,15470,toto@yopmail.fr
1,15168,a@yopmail.fr
2,401,nico@outlook.fr


In [19]:
columns2 = ['idUser',"mail","Number of action","Segment"]
liste = []
for id_user in list(df2['idUser']):
	df_row = df2[df2['idUser'] == id_user]
	dataBase_data_row = dataBase_data[dataBase_data['IdUser'] == id_user]
	Nb_of_action = df_row.iloc[0]["Number of typeAction"]
	segment = df_row.iloc[0]["Segment"]
	if len(dataBase_data_row) != 0:
		liste.append((id_user,dataBase_data_row.iloc[0]['mail'],Nb_of_action,segment))
	else:
		liste.append((id_user,"No mail",Nb_of_action,segment))
df_associaton_mail = pd.DataFrame(liste, columns = columns2)

In [21]:
#j'afficherai que les  10 premiàres lignes à titre d'exemple
df_associaton_mail.head(10)

,idUser,mail,Number of action,Segment
0,9083,utilisateur1@outlook.fr,1,1
1,8,tom@yopmail.fr,1,1
2,401,nico@outlook.fr,19,5
3,19,yolo@outlook.fr,2,1
4,13846,batman@gmail.com,1,1
5,15153,eric@yopmail.fr,21,5
6,15158,olivier@yopmail.fr,1,1
7,8381,user123@mail.fr,12,5
8,15168,a@yopmail.fr,3,2
9,15428,harold@outlook.fr,6,5


In [23]:
#Enregistrement sur dans un fichier csv
#Nous allons enregistrer que les 3 colonnes demandées
df3 = df_associaton_mail[["mail","Number of action","Segment"]]
df3.to_csv('data_output.csv', sep='\t', encoding='utf-8')